In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import nltk
from nltk.corpus import stopwords 

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential
from tensorflow.keras import optimizers

# Read data

In [ ]:
train_data = pd.read_csv(r'/kaggle/input/nlp-getting-started/train.csv', encoding='latin_1')
train_data.info()
train_data.head()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_x = train_data['text'].copy()
train_y = train_data['target'].copy()

In [ ]:
train_x.head()

In [ ]:
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(12,6)})
sns.countplot(train_y)

# clean data

In [ ]:
stop = stopwords.words('english')
def clean(text):
    
    text = re.sub(r'http\S+', ' ', text)
    
    text = re.sub(r'<.*?>', ' ', text)    
    
    text = re.sub(r'#\w+', ' ', text)    
     
    text = re.sub(r'@\w+', ' ', text)
    
    text = re.sub(r'\d+', ' ', text)
    
    text = text.split()
    
    text = ' '.join([word for word in text if word not in stop])
    
    return text

In [ ]:
train_x_cleaned = train_x.apply(clean)
train_x_cleaned.head()

In [ ]:
max_len = max(train_x_cleaned.apply(len))
print('max length: {}'.format(max_len))

# Tokenize

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x_cleaned)
vocab_size = len(tokenizer.word_index) + 1
x = tokenizer.texts_to_sequences(train_x_cleaned)
x = pad_sequences(x, max_len, padding='post')
y = train_y
print('train_x_clean:', train_x_cleaned[4])
print('*'*50)
print('x:',x[5])
print('vocabulary size:{}'.format(vocab_size))

In [ ]:
print(x.shape, y.shape)


# model

In [ ]:

epoch_size =10
batch_size = 32
embedding_dim = 16
optimizer = optimizers.Adam(lr=3e-4)

model = Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    layers.Bidirectional(layers.GRU(256, return_sequences=True)),
    layers.GlobalMaxPool1D(),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(2, activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.fit(x, y, epochs=epoch_size, validation_split=0.1)

In [ ]:
test_data = pd.read_csv(r'/kaggle/input/nlp-getting-started/test.csv', encoding='latin_1')
test_data.head()

In [ ]:
test_x = test_data['text'].copy()
test_x = test_x.apply(clean)
test_x = tokenizer.texts_to_sequences(test_x)
test_x = pad_sequences(test_x, max_len, padding='post')

In [ ]:
test_pred = np.argmax(model.predict(test_x), axis=1)
print(test_pred)

In [ ]:
submission = pd.DataFrame({'id':test_data['id'], 'target':test_pred})
submission.to_csv('submission.csv', index=False)